# Import Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import csv
import warnings
warnings.filterwarnings("ignore")

# PATH to CSV File

In [ ]:
# CSV file with the URLs
csv_file_path = '/Users/sachinsrivastava/Downloads/PythonCodes_Local/Hass_DataInnovationLab/DimensionsTask_06172023/mnsc_citationurl.csv'

# Initiate Selenium AND LOGIN STEP: YOU NEED TO FINISH LOGIN IN 60 SECONDS

In [ ]:
# Path to your Chromedriver executable
path_to_chromedriver = '/Users/sachinsrivastava/Downloads/chromedriver'

# Initiate Selenium webdriver
driver = webdriver.Chrome(executable_path=path_to_chromedriver)

# Login page URL
login_url = 'https://app.dimensions.ai/login' # Update with actual login page URL

# Open the login page
driver.get(login_url)

# Wait for login to complete
time.sleep(60)

# SET THE INDEX YOU WISH TO START THE LOOP FROM

In [ ]:
# Starting index
start_index = 0

# MAIN LOOP

In [ ]:
with open(csv_file_path, newline='') as f:
    reader = csv.DictReader(f)
    urls = [row['dimensions_citationurl'] for row in reader]

output_data = []

for idx, url in enumerate(urls[start_index:], start=start_index):
    print(f'Processing row index: {idx}')
    success_flag = "Success"

    # Open the URL
    driver.get(url)

    # Let the page load
    time.sleep(5)

    # Get the publication count
    try:
        publication_count_element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@data-bt="result_count_publication_plus"]')))
        publication_count = publication_count_element.text
        print(publication_count)
    except TimeoutException:
        publication_count = -1
        print("Could not retrieve publication count, set to -1.")
        success_flag = "Fail"

    # Click on the 'Save / Export' button
    try:
        save_export_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[normalize-space()='Save / Export']")))
        save_export_button.click()
    except TimeoutException:
        print("Could not click on 'Save / Export' button.")
        success_flag = "Fail"

    # Let the page load
    time.sleep(5)

    # Click on the 'Export results' button
    try:
        export_results_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[normalize-space()='Export results']")))
        export_results_button.click()
    except TimeoutException:
        print("Could not click on 'Export results' button.")
        success_flag = "Fail"

    # Let the page load
    time.sleep(5)

    # Click on the final 'Export' button in the modal
    try:
        final_export_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[normalize-space()='Export']")))
        final_export_button.click()
    except TimeoutException:
        print("Could not click on final 'Export' button.")
        success_flag = "Fail"

    # Let the page load
    time.sleep(5)

    # Append the URL and the publication count to the output data
    output_data.append([url, publication_count, success_flag])

    # Save to a new CSV file every 50 URLs
    if (idx + 1) % 50 == 0:
        with open(f'output_data_{(idx + 1)//100}.csv', 'w', newline='') as output_file:
            writer = csv.writer(output_file)
            writer.writerow(['URL', 'Publication_Count', 'Success/Fail?'])
            writer.writerows(output_data)
            # Reset the output data
            output_data = []

# Save the remaining data if there's any
if output_data:
    with open(f'output_data_{len(urls)//100 + 1}.csv', 'w', newline='') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['URL', 'Publication_Count', 'Success/Fail?'])
        writer.writerows(output_data)

# Close the webdriver
driver.quit()